In [1]:
import csv
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
from scipy.optimize import root, fsolve
from scipy.special import binom
import pandas as pd

In [54]:
p_a = 1
T_max = 12
D_max = 1
W_max = D_max
O_max = D_max

Nr = 3
nodes = 20

### Formulas for exponential backoff

In [73]:
def pi_idle(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    elif p_f == 0.:
        p_f = 0.0000001
    total_sum = 0
    for i in range(1, Nr + 1 + 1):
        first_addition = W_max + 1 / (1 - p_f)
        numerator = (2**i * T_max + 1) * (p_f ** (2**i * T_max + 1) - 2**i * T_max * p_f - p_f + 2**i * T_max)
        denominator = 2**i * T_max * (1 - p_f ** (2**i * T_max + 1))
        second_addition = (1 - p) * (2 + D_max)
#         huge_sum = 1 + p / p_f + first_addition * numerator / denominator + second_addition
        huge_sum = 1 + p * ((1 - p_f ** O_max) / (p_f ** O_max * (1 - p_f))) + first_addition * numerator / denominator + second_addition
        total_sum += p_a * p ** (i - 1) * huge_sum
    return (1 + total_sum) ** (-1)

In [71]:
pi_idle(0.4, 0.8)

0.005854819088338644

In [57]:
def pi_rts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        total_sum += p ** (i - 1) * p_a * pi_idle(p, p_f)
    return total_sum

# def pi_out(p, p_f):
#     total_sum = 0
#     for i in range(1, Nr + 1 + 1): 
#         total_sum += p ** (i - 1) * p_a * p * pi_idle(p, p_f) / p_f
#     return total_sum

def pi_out(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        total_sum += p_a * p ** (i - 1) * p * pi_idle(p, p_f) * ((1 - p_f ** O_max) / (p_f ** O_max * (1 - p_f)))
    return total_sum

def pi_cts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        total_sum += (1 - p) * p ** (i - 1) * p_a * pi_idle(p, p_f)
    return total_sum

def pi_bo(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * pi_idle(p, p_f) * (2**i * T_max + 1) * (p_f ** (2**i * T_max + 1) - 2**i * T_max * p_f - p_f + 2**i * T_max)
        denominator = (2**i * T_max) * (1 - p_f) * (1 - p_f ** (2**i * T_max + 1))
        total_sum += numerator / denominator
    return total_sum

def pi_wait(p, p_f):
    if p_f == 1.:
        p_f = 0.9999999
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        numerator = p ** (i - 1) * p_a * pi_idle(p, p_f) * (2**i * T_max + 1) * (p_f ** (2**i * T_max + 1) - 2**i * T_max * p_f - p_f + 2**i * T_max)
        denominator = (2**i * T_max) * (1 - p_f ** (2**i * T_max + 1))
        total_sum += numerator / denominator
    return W_max * total_sum

def pi_cts(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        total_sum += (1 - p) * p ** (i - 1) * p_a * pi_idle(p, p_f)
    return total_sum
    
def pi_data(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        total_sum += (1 - p) * p ** (i - 1) * p_a * pi_idle(p, p_f)
    return D_max * total_sum

def pi_ack(p, p_f):
    total_sum = 0
    for i in range(1, Nr + 1 + 1): 
        total_sum += (1 - p) * p ** (i - 1) * p_a * pi_idle(p, p_f)
    return total_sum

In [58]:
def p_success(p):
    total_sum = 0
    for i in range(1, Nr + 1 + 1):
        total_sum += (1 - p) * p ** (i - 1) * p_a
    return total_sum

# alternative formula
def p_success(p):
    return 1 - p ** (Nr + 1)

def p_failure(p):
    return p ** (Nr + 1)

In [59]:
def transcendental_equation_p_collision(p, p_f, n):
    p_rts = pi_rts(p, p_f)
    p_cts = pi_cts(p, p_f)
    p_data = pi_data(p, p_f)
    p_ack = pi_ack(p, p_f)
    # We solve transcendental equation f(x) = g(x) by turning it into f(x) - g(x) = 0
#     return p - (1 - (1 - p_rts) ** (n) - n * p_rts * (1 - p_rts) ** (n-1))
#     return p - (1 - (1 - p_rts) ** (2*(n-1))) - (1 - (1 - p_cts) ** (2*(n-1)))
    return p - (1 - (1 - p_rts) ** ((n-1))) - (1 - (1 - p_cts) ** ((n-1)))

def transcendental_equation_p_free(p, p_f, n):
    p_rts = pi_rts(p, p_f)
    p_cts = pi_cts(p, p_f)
    p_data = pi_data(p, p_f)
    p_ack = pi_ack(p, p_f)
    return p_f - (1 - p_cts) ** (n-1)

def system_of_equations(p_pf, node):
    p, pf = p_pf
    return (transcendental_equation_p_collision(p, pf, node), transcendental_equation_p_free(p, pf, node))

def calculate_p_pf_from_system():
    p_array = []
    pf_array = []
    for node in range(1, nodes+1):
        p, pf = fsolve(system_of_equations, (0.01, 0.01), args=(node))
        p_array.append(p)
        pf_array.append(pf)
    return p_array, pf_array

In [60]:
p_array, p_f_array = calculate_p_pf_from_system()

In [61]:
p_array

[1.8086627663754192e-27,
 0.08660267093728163,
 0.14107498807466884,
 0.18190358901228282,
 0.21513477457816488,
 0.24343439132373987,
 0.26823109930689076,
 0.29038998623134904,
 0.31047956574086233,
 0.32889589339737035,
 0.3459271845604028,
 0.36179040464041057,
 0.37665337013848166,
 0.3906487973386866,
 0.40388360969624076,
 0.41644531863697287,
 0.4284065256425171,
 0.439828177680964,
 0.45076197160638876,
 0.46125216316011325]

In [62]:
p_f_array

[1.0,
 0.9586585352021165,
 0.9347183006517712,
 0.9178631164965532,
 0.9048573404099832,
 0.8943018020625592,
 0.8854572417487248,
 0.8778815440602455,
 0.8712874852956345,
 0.8654771291070398,
 0.8603079065069665,
 0.8556735336286302,
 0.8514925592677688,
 0.8477011309465111,
 0.8442482322820137,
 0.8410924380331061,
 0.8381996384606334,
 0.8355414036256031,
 0.8330937823329605,
 0.830836403636758]

In [72]:
1 / pi_idle(0.46125216316011325, 0.830836403636758)

196.72307544910802

In [11]:
prt = pi_rts(p_array[9], p_f_array[9])
prt

0.023728874830785954

In [12]:
1 - (1 - prt) ** (10) - 10 * prt * (1 - prt) ** (9)

0.022323427120735456

In [13]:
(1 - prt) ** (10)

0.786510355786292

In [14]:
10 * prt * (1 - prt) ** (9)

0.19116621709297252

In [15]:
analytics_headers = [
    'nodes',
    'p_collision',
    'p_success',
    'p_failure',
    'p_free',
    'cycle_time',
    'p_bo',
    'p_wait',
    'p_rts',
    'p_out',
    'p_cts',
    'p_data',
    'p_ack',
    'bo_time',
    'wait_time',
    'rts_time',
    'out_time',
    'cts_time',
    'data_time',
    'ack_time',
]

In [86]:
data = [analytics_headers]
for i in range(nodes):
    data.append([
        i + 1,
        p_array[i],
        p_success(p_array[i]),
        p_failure(p_array[i]),
        p_f_array[i],
        Etc(p_array[i], p_f_array[i]),
        pi_bo(p_array[i], p_f_array[i]),
        pi_wait(p_array[i], p_f_array[i]),
        pi_rts(p_array[i], p_f_array[i]),
        pi_out(p_array[i], p_f_array[i]),
        pi_cts(p_array[i], p_f_array[i]),
        pi_data(p_array[i], p_f_array[i]),
        pi_ack(p_array[i], p_f_array[i]),
        pi_bo(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_wait(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_rts(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_out(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_cts(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_data(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
        pi_ack(p_array[i], p_f_array[i]) * Etc(p_array[i], p_f_array[i]),
    ])

In [87]:
# Draw table to evaluate the results
data_pd = {}
for i in range(1, len(data[0])):
    data_pd[data[0][i]] = np.array(data[1:]).T[i]
df1 = pd.DataFrame(data_pd, index=np.arange(1, len(np.array(data[1:]).T[0])+1))
df1

,p_collision,p_success,p_failure,p_free,cycle_time,p_bo,p_wait,p_rts,p_out,p_cts,p_data,p_ack,bo_time,wait_time,rts_time,out_time,cts_time,data_time,ack_time
1,1.808663e-27,1.000000,1.070115e-107,1.000000,52.525587,0.714425,7.144248e-08,0.057115,1.033018e-28,0.057115,0.057115,0.057115,37.525583,0.000004,3.000000,5.425988e-27,3.000000,3.000000,3.000000
2,8.660267e-02,0.999944,5.625034e-05,0.958659,72.562287,0.754106,3.117585e-02,0.045261,4.088777e-03,0.041341,0.041341,0.041341,54.719655,2.262191,3.284257,2.966910e-01,2.999831,2.999831,2.999831
3,1.410750e-01,0.999604,3.960957e-04,0.934718,90.348258,0.772326,5.041873e-02,0.038643,5.832348e-03,0.033192,0.033192,0.033192,69.778281,4.555245,3.491355,5.269425e-01,2.998812,2.998812,2.998812
4,1.819036e-01,0.998905,1.094876e-03,0.917863,106.399434,0.781842,6.421806e-02,0.034427,6.822838e-03,0.028165,0.028165,0.028165,83.187542,6.832765,3.663034,7.259461e-01,2.996715,2.996715,2.996715
5,2.151348e-01,0.997858,2.142113e-03,0.904857,121.272338,0.787367,7.491223e-02,0.031451,7.477624e-03,0.024685,0.024685,0.024685,95.485883,9.084781,3.814124,9.068289e-01,2.993574,2.993574,2.993574
6,2.434344e-01,0.996488,3.511783e-03,0.894302,135.302635,0.790804,8.358652e-02,0.029204,7.949474e-03,0.022095,0.022095,0.022095,106.997818,11.309477,3.951362,1.075585e+00,2.989465,2.989465,2.989465
7,2.682311e-01,0.994823,5.176504e-03,0.885457,148.695466,0.793038,9.083674e-02,0.027428,8.308778e-03,0.020071,0.020071,0.020071,117.921132,13.507012,4.078433,1.235478e+00,2.984470,2.984470,2.984470
8,2.903900e-01,0.992889,7.110932e-03,0.877882,161.583728,0.794533,9.702714e-02,0.025978,8.593109e-03,0.018434,0.018434,0.018434,128.383601,15.678007,4.197612,1.388507e+00,2.978667,2.978667,2.978667
9,3.104796e-01,0.990708,9.292490e-03,0.871287,174.058276,0.795552,1.023975e-01,0.024764,8.824629e-03,0.017075,0.017075,0.017075,138.472363,17.823126,4.310420,1.536000e+00,2.972123,2.972123,2.972123
10,3.288959e-01,0.988299,1.170129e-02,0.865477,186.184036,0.796253,1.071142e-01,0.023729,9.017372e-03,0.015925,0.015925,0.015925,148.249563,19.942957,4.417938,1.678891e+00,2.964896,2.964896,2.964896


In [88]:
# Here we save our table to csv in the same folder
with open('2021-04-08-01.csv', "wt", newline="") as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(analytics_headers)
    for i in range(1, len(data)):
        writer.writerow(data[i])

In [17]:
s = 0
for i in range(1, 21):
    s += i
    print(s)

1
3
6
10
15
21
28
36
45
55
66
78
91
105
120
136
153
171
190
210


In [18]:
s

210

In [73]:
simpc = 0.4663487125083911
simpf = 0.7931865846302574
t1 = pi_idle(simpc, simpf) + pi_bo(simpc, simpf) + pi_rts(simpc, simpf)
t2 = pi_cts(simpc, simpf) + pi_data(simpc, simpf) + pi_ack(simpc, simpf)
t3 = pi_out(simpc, simpf) + pi_wait(simpc, simpf)
t1 + t2 + t3

1.0

In [71]:
vc = 0.4731246508351256
vf = 0.8260981839009207
Etc(vc, vf)

307.9090475329297

In [29]:
s = 0
for i in range(9, 13):
    s += i
print(s)

42


In [30]:
s = 0
for i in range(13, 16):
    s += i
print(s)

42


In [31]:
s = 0
for i in range(16, 18):
    s += i
print(s)

33


In [32]:
s = 0
for i in range(18, 20):
    s += i
print(s)

37


In [33]:
s = 0
for i in range(20, 21):
    s += i
print(s)

20


In [61]:
cp = 0.11111884767821734
cf = 0.957330465104775

In [65]:
Etc(cp, cf) * pi_out(cp, cf)

0.3916852250936333

In [ ]:
1 - 8
9 - 12
13 - 15
16 - 17
18 - 19
20